<a href="https://colab.research.google.com/github/jbinteam/010723305/blob/main/Homework9p2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import cv2 
import scipy
import matplotlib.pylab as plt
from skimage import io

In [3]:
def preprocessing(img) :
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return (img, img_gray)

In [4]:
sift = cv2.SIFT_create()
bf = cv2.BFMatcher()

In [5]:
def augmented_image(frame,im_src, pts_src, pts_dst):
    
    # Calculate Homography
    h, status = cv2.findHomography(pts_src, pts_dst)

    # Warp source image to destination based on homography
    warped_image = cv2.warpPerspective(im_src, h, (frame.shape[1],frame.shape[0]))
            
    # Prepare a mask representing region to copy from the warped image into the original frame.
    mask = np.zeros([frame.shape[0], frame.shape[1]], dtype=np.uint8)
    cv2.fillConvexPoly(mask, np.int32(pts_dst), (255, 255, 255), cv2.LINE_AA)
    im_out = cv2.add(frame, warped_image, mask=cv2.bitwise_not(mask))
    im_out = cv2.add(im_out, warped_image)
    
    # cv2.imshow('augmented', im_out)

    markerCorners, markerIds, rejectedCandidates = cv2.aruco.detectMarkers(im_out, AruCo_dict, parameters = AruCo_params)

    if len(markerCorners) > 0:
            rvecs, tvecs, points = cv2.aruco.estimatePoseSingleMarkers(markerCorners , 0.1, K, dist)
            for (rvec, tvec, id, corner) in zip(rvecs, tvecs, markerIds, markerCorners) :
                x = tvec[0,0]
                y = tvec[0,1]
                z = tvec[0,2]
                text = "X : {:.2f}  Y : {:.2f}  Z : {:.2f}".format(x, y, z)
                X = (corner[0,0][0] + corner[0,2][0]) / 2
                Y = (corner[0,0][1] + corner[0,2][1]) / 2
            return X, Y, text
    else:
        return 0,0 ,""
    

In [6]:
def feature_object_detection(template_img, template_gray, query_img, query_gray, min_match_number) :

    img = cv2.imread('./4x4_1000.png')
    im_src_size = img.shape[:2]
    # src_points = np.float32([[0,0], [im_src_size[1],0],[im_src_size[1], im_src_size[0]] ,[0, im_src_size[0]] ])
    src_points = np.float32([[0,0], [0, im_src_size[0]],[im_src_size[1], im_src_size[0]] ,[im_src_size[1],0] ])

    template_kpts, template_desc = sift.detectAndCompute(template_gray, None)
    query_kpts, query_desc = sift.detectAndCompute(query_gray, None)
    matches = bf.knnMatch(template_desc, query_desc, k=2)
    good_matches = list()
    good_matches_list = list()

    for m, n in matches :
        if m.distance < 0.6*n.distance :
            good_matches.append(m)
            good_matches_list.append([m])
    
    if len(good_matches) > min_match_number :
        src_pts = np.float32([ template_kpts[m.queryIdx].pt for m in good_matches ]).reshape(-1,1,2)
        dst_pts = np.float32([ query_kpts[m.trainIdx].pt for m in good_matches ]).reshape(-1,1,2)

        H, inlier_masks = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 10) # H RANSAC
        # get the bounding box around template image 
        h, w = template_img.shape[:2]
        template_box = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1,1,2)
        transformed_box = cv2.perspectiveTransform(template_box, H)
        
        X, Y, T = augmented_image(query_img, img, src_points,  transformed_box)
        

        detected_img = cv2.polylines(query_img, [np.int32(transformed_box)], True, (255,0,0), 3, cv2.LINE_AA)
        drawmatch_img = cv2.drawMatchesKnn(template_img, template_kpts, detected_img, query_kpts, good_matches_list, None, flags=2, matchesMask=inlier_masks)
        
        write_text(detected_img, (X, Y), 20, T)
        cv2.imshow('frame',   drawmatch_img )
        
        return
    else :
        print('Keypoints not enough')
        return
    

In [7]:
params_dir = os.getcwd()+'./new-camera_params/'
print(params_dir)

#load camera parameters
K = np.load(params_dir+'K.npy')
dist = np.load(params_dir+'dist.npy')

print("Camera matrix")
print(K)
print("Len distortion")
print(dist)

d:\3 first semester\img\Final-examination./new-camera_params/
Camera matrix
[[726.47630281   0.         466.2575863 ]
 [  0.         732.06406134 251.99786765]
 [  0.           0.           1.        ]]
Len distortion
[[ 0.02450981  0.03012141  0.00045661  0.01036187 -0.07354295]]


In [8]:
AruCo_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_4X4_1000)
AruCo_params = cv2.aruco.DetectorParameters_create()
# board = cv2.aruco.GridBoard_create(3, 4, 0.05, 0.0075, AruCo_dict)

In [9]:
def write_text(img, pose, dy, text) :
    x0 = pose[0]
    y0 = pose[1]
    for i, line in enumerate(text.split('\n')) :
        y = y0 + i*dy
        cv2.putText(img, line, np.int32([x0, y]), cv2.FONT_HERSHEY_COMPLEX, 0.75, (50,200,255), 2)

In [10]:
def fram_gray_real(frame,frame_gray) :
    frame_hsv = cv2.cvtColor(frame,cv2.COLOR_RGB2HSV)

    kernel =np.array([[255,255,255],[255,255,255],[255,255,255]],np.uint8)
    
    

    upper_range_red=np.array([125,255,255]) 
    lower_range_red=np.array([[110,50,50]]) 

    mask_red = cv2.inRange(frame_hsv,lower_range_red,upper_range_red)
    

    upper_range_yellow=np.array([98,255,255]) 
    lower_range_yellow=np.array([[90,80,80]]) 

    mask_yellow = cv2.inRange(frame_hsv,lower_range_yellow,upper_range_yellow)


    mask = cv2.add( mask_red,mask_yellow)
    mask = cv2.erode(mask,kernel,iterations=2)
    mask = cv2.dilate(mask,kernel,iterations=8)
    mask_indices=np.where(mask==255)

    output_image = np.zeros(frame_gray.shape, dtype = "uint8")
    output_image[mask_indices] = frame_gray[mask_indices] 
    
    # cv2.imshow('frame1', mask)
    cv2.imshow('frame2', output_image)
    # cv2.imshow('frame4', frame)
    
    return output_image

In [11]:
template_img = cv2.imread('./Template-3.png')
template_img, template_gray = preprocessing(template_img)

def farneback_dense_optical_flow(video_device) :
    cap = cv2.VideoCapture(video_device)

    while cap.isOpened() :
        ret, frame = cap.read()

        if ret :
            
            frame_gray1 = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            frame_gray2 = fram_gray_real(frame,frame_gray1)

            # cv2.imshow('frame4',  frame_gray2)
            feature_object_detection(template_img, template_gray, frame, frame_gray2, 10)
            # cv2.imshow('frame',   frame )
            key = cv2.waitKey(27) & 0xFF
            if key == 27 or key == ord('q') :
                break
        else :
            break
    
    cap.release()
    cv2.destroyAllWindows()

farneback_dense_optical_flow('./right_output.avi')

Keypoints not enough
Keypoints not enough
Keypoints not enough
Keypoints not enough
Keypoints not enough
Keypoints not enough
Keypoints not enough
Keypoints not enough
Keypoints not enough
Keypoints not enough
Keypoints not enough
Keypoints not enough
Keypoints not enough
Keypoints not enough
Keypoints not enough
Keypoints not enough
Keypoints not enough
Keypoints not enough
Keypoints not enough
Keypoints not enough
Keypoints not enough
Keypoints not enough
Keypoints not enough
Keypoints not enough


error: OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-u4kjpz2z\opencv\modules\core\src\matmul.dispatch.cpp:531: error: (-215:Assertion failed) scn + 1 == m.cols in function 'cv::perspectiveTransform'
